<a href="https://colab.research.google.com/gist/hathibelagal-dev/6f2d35a1bf01b2222a0e57cf5c493416/deepseekocr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Optional
This is for word-wrapping the output of OCR process

In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
      white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

### Setup

In [ ]:
!pip3 install --no-deps -q bitsandbytes

In [ ]:
!pip3 install addict transformers==4.46.3 tokenizers==0.20.3 pdf2image

In [ ]:
!apt install poppler-utils

In [ ]:
!mkdir -p /content/outputs /content/pdf_pages/

### PDF to Images

In [ ]:
from pdf2image import convert_from_path

pdf_file = 'test.pdf'
images = convert_from_path(pdf_file)

for i, image in enumerate(images):
    image.save(f'/content/pdf_pages/page_{i+1}.jpg', 'JPEG')

In [ ]:
from IPython.display import Image as _I
_I("/content/pdf_pages/page_1.jpg", width=640)

### OCR

In [ ]:
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig
import torch

model_name = 'deepseek-ai/DeepSeek-OCR'

In [ ]:
qc = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(
    model_name, trust_remote_code=True,
    use_safetensors=True, device_map="auto",
    quantization_config=qc, torch_dtype=torch.float
)
model = model.eval()

In [ ]:
page_number = 5 #@param {type:"integer"}

prompt = "<image>\nParse the figure."
image_file = f'/content/pdf_pages/page_{page_number}.jpg'
output_path = f'/content/outputs/page_{page_number}'

In [ ]:
%%time
model.infer(tokenizer, prompt=prompt, image_file=image_file, output_path = output_path, base_size = 1024, image_size = 1024, crop_mode=False, save_results = True, test_compress = True)